In [1]:
import numpy as np

In [2]:
cls_0 = [
    "a b b a b c",
    "a c b c b b b a",
    "b a b a b c",
]

cls_1 = [
    "d e b a b c",
    "d c b d b b e a",
    "a a c e b c",
    "d a c a c c e e",
]

In [3]:
s = "a b c a b b x a"

In [4]:
def build_w2i(cls_list):
    w2i = {
        "<unk>": 0
    }
    i = 1
    for cls in cls_list:
        for s in cls:
            tokens = s.split()
            for token in tokens:
                if token in w2i:
                    continue

                w2i[token] = i
                i += 1
    return w2i

In [5]:
w2i = build_w2i([cls_0, cls_1])
print(w2i)

{'<unk>': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}


In [6]:
len(w2i)

6

In [7]:
def build_pi(w2i, docs):
    pi = np.ones((len(w2i), ))
    for s in docs:
        tokens = s.split()
        first_token = tokens[0]
        first_token_index = w2i[first_token]
        pi[first_token_index] += 1

    pi = pi / pi.sum()
    return pi

In [8]:
pi_0 = build_pi(w2i, cls_0)
pi_1 = build_pi(w2i, cls_1)

In [9]:
def build_A(w2i, docs):
    A = np.ones((len(w2i), len(w2i)))
    for doc in docs:
        tokens = doc.split()
        token_count = len(tokens)
        for i in range(1, token_count):
            curr = tokens[i]
            prev = tokens[i-1]
            curr_token_index = w2i[curr]
            prev_token_index = w2i[prev]
            A[prev_token_index][curr_token_index] += 1

    A = A / A.sum(axis=1, keepdims=True)
    return A

In [10]:
A_0 = build_A(w2i, cls_0)
A_1 = build_A(w2i, cls_1)

In [11]:
# cls 0'a ait pi ve A1
print("pi_0:", pi_0)
print("A_0:", A_0)

print("-" * 70)

# cls 1'e ait pi ve A1
print("pi_1:", pi_1)
print("A_1:", A_1)

pi_0: [0.11111111 0.33333333 0.22222222 0.11111111 0.11111111 0.11111111]
A_0: [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.09090909 0.09090909 0.45454545 0.18181818 0.09090909 0.09090909]
 [0.0625     0.3125     0.25       0.25       0.0625     0.0625    ]
 [0.125      0.125      0.375      0.125      0.125      0.125     ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
----------------------------------------------------------------------
pi_1: [0.1 0.2 0.1 0.1 0.4 0.1]
A_1: [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.09090909 0.18181818 0.18181818 0.36363636 0.09090909 0.09090909]
 [0.08333333 0.16666667 0.16666667 0.25       0.16666667 0.16666667]
 [0.09090909 0.18181818 0.18181818 0.18181818 0.09090909 0.27272727]
 [0.1        0.2        0.2        0.2        0.1        0.2       ]
 [0.1        0.2        0.3        0.1        0.1    

In [12]:
print(w2i, "\n", s)

{'<unk>': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5} 
 a b c a b b x a


In [13]:
def get_w2i(token, w2i):
    if token in w2i:
        return w2i[token]
    else:
        return w2i["<unk>"]

In [14]:
def calc_likelihood(s, pi, A, w2i):
    tokens = s.split()
    total_proba = 1.0
    for i, token in enumerate(tokens):
        token_index = get_w2i(token, w2i)
        if i == 0:
            proba = pi[token_index]
            total_proba *= proba
        else:
            previous_token = tokens[i-1]
            previous_token_index = get_w2i(previous_token, w2i)
            proba = A[previous_token_index][token_index]
            total_proba *= proba

    return total_proba

In [15]:
likelihood_0 = calc_likelihood(s, pi_0, A_0, w2i)
likelihood_1 = calc_likelihood(s, pi_1, A_1, w2i)
print(likelihood_0, likelihood_1)

5.604697543617997e-06 6.956618526866461e-07


In [16]:
prior_0 = len(cls_0) / (len(cls_0) + len(cls_1))
prior_1 = len(cls_1) / (len(cls_0) + len(cls_1))
print(prior_0, prior_1)

0.42857142857142855 0.5714285714285714


In [18]:
proba_0 = prior_0 * likelihood_0
proba_1 = prior_1 * likelihood_1
if proba_0 > proba_1:
    print(f"'{s}' dizisi ilk sınıfa (cls_0) aittir")
else:
    print(f"'{s}' dizisi ikinci sınıfa (cls_1) aittir")

'a b c a b b x a' dizisi ilk sınıfa (cls_0) aittir
